In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms

import time
import numpy as np
import os
from tqdm import  tqdm

In [2]:
N_INPUTS = 32 * 32 * 3
N_CLASSES = 10
HIDDEN_LAYER = '64, 64, 64'

# Default constants
LEARNING_RATE = 0.01
EPOCH = 1500
EVAL_FREQ = 10

dataset_size = 1000
TEST_SIZE = 0.2

BATCH_SIZE = 128

In [3]:
transform_train = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
 
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4941, 0.4850, 0.4502), (0.2467, 0.2429, 0.2616))])
    
    
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train)

trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2)
 
testset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform_test)

testloader = torch.utils.data.DataLoader(
    testset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2)
 
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class MLP(nn.Module):

    def __init__(self, n_inputs, n_hidden, n_classes):
        """
        Initializes multi-layer perceptron object.    
        Args:
            n_inputs: number of inputs (i.e., dimension of an input vector).
            n_hidden: list of integers, where each integer is the number of units in each linear layer
            n_classes: number of classes of the classification problem (i.e., output dimension of the network)
        """
        super(MLP, self).__init__()
        self.input_layers = nn.Sequential()
        self.hidden_layers = nn.Sequential()
        self.output_layers = nn.Sequential()
        for index in range(len(n_hidden)):
            if index == 0:
                self.input_layers.add_module("input_layer",
                                             nn.Linear(in_features=n_inputs, out_features=int(n_hidden[index]),
                                                       bias=True))
                self.input_layers.add_module("input_relu",
                                             nn.ReLU())
            if index == len(n_hidden) - 1:
                self.output_layers.add_module("output_layer",
                                              nn.Linear(in_features=int(n_hidden[index]), out_features=n_classes,
                                                        bias=True))
            else:
                self.hidden_layers.add_module("hidden_layer" + str(index),
                                              nn.Linear(in_features=int(n_hidden[index]),
                                                        out_features=int(n_hidden[index + 1]), bias=True))
                self.hidden_layers.add_module("hidden_relu" + str(index),
                                              nn.ReLU())
                
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.001)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """
        Predict network output from input by passing it through several layers.
        Args:
            x: input to the network
        Returns:
            out: output of the network
        """
        x = x.view(-1,32 * 32 * 3)    
        x = self.input_layers(x)
        x = self.hidden_layers(x)
        x = self.output_layers(x)
        return x


In [5]:
loss_train_list = []
loss_test_list = []
acc_train_list = []
acc_test_list = []
epoch_list = []

In [6]:
def train(optimizer, learning_rate, max_epochs, eval_freq, ada_opt, save_para):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
    print("start training CNN...")
    print(device)
    # 储存学习到的参数
    opt_save_path = "optimizer" + optimizer + '.pth'
    writer = SummaryWriter('./logs-MLP-image/' + optimizer + '/lr=' + str(learning_rate))
    
    hidden_layer = HIDDEN_LAYER.split(",")
    model = MLP(n_inputs=N_INPUTS, n_hidden=hidden_layer, n_classes=N_CLASSES)
    model.to(device)
    
    cost = nn.CrossEntropyLoss()
    
    if optimizer == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer == 'ADAM':
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer == 'RMSprop':
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    
    if ada_opt:
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80, 100, 120], gamma=0.1)
    
    for epoch in tqdm(range(max_epochs)):
        
        model.train()
        
        train_acc_avg = 0.
        train_loss_avg = 0.
        
        for idx, (image, label) in enumerate(trainloader):
            
            image, label = image.to(device), label.to(device)
            
            optimizer.zero_grad()
            output = model(image)
            loss = cost(output, label)
            loss.backward()
            optimizer.step()
            
            train_loss_avg = train_loss_avg + loss.item()
            
            _, pred = output.max(1)
            num_correct = (pred == label).sum().item()
            train_acc_avg = train_acc_avg + num_correct/label.shape[0]
            
        train_loss_avg = train_loss_avg/len(trainloader)
        train_acc_avg = train_acc_avg/len(trainloader)
        print("epoch is %d, train_loss is %lf, train_acc is %lf" %(epoch+1, train_loss_avg, train_acc_avg))
        writer.add_scalar('Train/Accuracy', train_acc_avg, epoch)
        writer.add_scalar('Train/Loss', train_loss_avg, epoch)
        if ada_opt: scheduler.step()
        
        test_acc_avg = 0.
        test_loss_avg = 0.
        
        with torch.no_grad():
            for idx, (image, label) in enumerate(testloader):
                
                image, label = image.to(device), label.to(device)
                
                output = model(image)
                loss = cost(output, label)
                
                test_loss_avg = test_loss_avg + loss.item()
                
                _, pred = output.max(1)
                num_correct = (pred == label).sum().item()
                test_acc_avg = test_acc_avg + num_correct/label.shape[0]
            
        test_loss_avg = test_loss_avg/len(testloader)
        test_acc_avg = test_acc_avg/len(testloader)
        print('epoch is %d, test_loss is %lf, test_acc is %lf' %(epoch+1, test_loss_avg, test_acc_avg))
        writer.add_scalar('Test/Accuracy', test_acc_avg, epoch)
        writer.add_scalar('Test/Loss', test_loss_avg, epoch)
    
    return model
    

In [7]:
#lr=1e-5 para defalut opt
model = train(optimizer='ADAM', learning_rate=1e-4, max_epochs=200, eval_freq=500, ada_opt=True, save_para=False)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 2.149992, train_acc is 0.168934


  0%|          | 1/200 [00:07<24:54,  7.51s/it]

epoch is 1, test_loss is 2.062653, test_acc is 0.187203
epoch is 2, train_loss is 2.043097, train_acc is 0.198270


...

epoch is 198, test_loss is 1.599611, test_acc is 0.476562
epoch is 199, train_loss is 1.043474, train_acc is 0.613863


100%|█████████▉| 199/200 [25:22<00:07,  7.64s/it]

epoch is 199, test_loss is 1.595750, test_acc is 0.474486
epoch is 200, train_loss is 1.043575, train_acc is 0.614142


100%|██████████| 200/200 [25:30<00:00,  7.65s/it]

epoch is 200, test_loss is 1.596326, test_acc is 0.475079
